
# IMPORT DATA


In [3]:
import pandas as pd 
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import RMSprop
from keras import backend as K
import tensorflow as tf
import io
import pandas as pd

In [4]:
train= pd.read_csv('동원산업통합_분류_1_RE.csv',encoding='cp949')
train

,청구서번호,No.,Subject,Machinery,Assembly,청구품목,Part No.1,Part No.2,key1,key2,...,창고입고수량,Control No.,입고창고,창고출고,창고출고수량,출고선박,출고운반선,선박입고,선박입고수량,완료 여부
0,ELS-BS-ESP-2004002,12,NO.1 A/E C18 MAJOR O/H PARTS,NO.1 GENERATOR ENGINE,323-6480 LINES GP-FUEL,SEAL-O-RING-STOR,7.00E-275,NaN,7.00E-275,COOLER,...,1,혜인 발주,BS,2020-09-19,1,BLO,본선 선적,NaN,0,NaN
1,OCA-BS-ESP-2010004,5,운전시간 대비 #2 AUX ENG TOP END O/H PARTS,NO.2 GENERATOR ENGINE,GASKET KIT,OIL COOLER & LINES,7.00E-275,NaN,7.00E-275,COOLER,...,1,혜인 발주,BS,2020-09-18,1,JBG,본선 선적,NaN,0,NaN
2,MIR-BS-ESP-2011002,119,운전시간 대비 #2 AUX ENG(CAT 3512C) MAJOR O/H PARTS,NO.2 GENERATOR ENGINE,285-8374 MANIFOLD GP-EXH,WASHER,7.00E-275,NaN,7.00E-275,COOLER,...,1,NaN,BS,2020-12-12,1,BON,SANWA FONTAINE,NaN,0,NaN
3,BLO-BS-ESP-2003002,16,#1 AUX ENG TOP END O/H PARTS,NO.1 GENERATOR ENGINE,159-8828 TURBO GP,BOLT-HIGH TEMP,7.00E-275,NaN,7.00E-275,COOLER,...,1,혜인 발주,BS,2021-06-16,1,MIR,체항수리 시 사용,NaN,0,NaN
4,DEO-BS-ESP-2105008,97,DEO-E-210512-01,NO.1 GENERATOR ENGINE,2N4727 INSTRUMNT PANEL GP,SEAL,7.00E-275,NaN,7.00E-275,COOLER,...,1,NaN,BS,NaN,0,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21079,BLO-BS-ESP-2006006,1,"BLO-F-ENGINE-200515-03 BOAT DAVIT CYL 외, AIRCO...",HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,HYD. CYLINDER_NET BOAT DAVIT,φ150 × φ85 × 695st,FF-SE-1CB150BB695ABD-YP2J,φ150 × φ85 × 695stFF-SE-1CB150BB695ABD-YP2J,CYLINDER,...,2,20-09-161,BW,NaN,0,NaN,NaN,NaN,0,NaN
21080,BLO-BS-ESP-2004008,1,BLO-F-ENGINE-200403-01 (NAVI DECK CRANE DERRIN...,HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,HYD. CYLINDER_N.D CRANE_BOOM_LIFTER,φ160 × φ112 × 1130st,S-2CA160A20.6N1130-1,φ160 × φ112 × 1130stS-2CA160A20.6N1130-1,CYLINDER,...,1,20-07-114,BW,2020-08-26,1,BLO,BLO 상가시 선적,NaN,0,NaN
21081,ELS-BS-ESP-1908002,2,ELS-F-ENG-190628-02,6M NET BOAT,ELECTRIC CLUTCH,PULLY,Φ305X5V-3,NaN,Φ305X5V-3,BELT,...,1,19-08-164,BW,2019-09-04,1,ELS,ONENESS,2019-10-02,1,완료
21082,BLO-BS-ESP-1906014,2,BLO-F-ENGINE-190617-01 (NET BOAT PARTS),7M NET BOAT,HAND HYDRAULIC STEERING SYSTEM,CYLINDER,Φ50 x S195 CYLINDER,NaN,Φ50 x S195 CYLINDER,CYLINDER,...,1,19-07-134,BW,2019-08-20,1,BLO,세이셀 컨테이너,2019-10-12,1,완료


In [ ]:
# 이거를 이렇게 주지 말고 처음부터 db에서 받아라
import pymysql
conn=pymysql.connect(host='***.com',port=3306,user='slows14tem',password='***',db='bigdata')
cursor = conn.cursor()
sql = "select * from original"
cursor.execute(sql)

In [5]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [6]:
train['Machinery_Encode']= label_encoder.fit_transform(train['Machinery'])
train['Assembly_Encode']= label_encoder.fit_transform(train['Assembly'])
train['청구품목_Encode']= label_encoder.fit_transform(train['청구품목'])
train['part1_Encode']= label_encoder.fit_transform(train['Part No.1'])
train['key2_Encode']=label_encoder.fit_transform(train['key2'])

In [7]:
columns= ['Machinery_Encode'  , 'Assembly_Encode' , '청구품목_Encode', 'part1_Encode','key2_Encode' ]
features= ['Machinery_Encode'  , 'Assembly_Encode' , '청구품목_Encode', 'part1_Encode' ]
target= ['key2_Encode']

data_Encode 에 해당 컬럼들을 넣는다.

In [8]:
data_Encode= train[columns]
data_Encode

,Machinery_Encode,Assembly_Encode,청구품목_Encode,part1_Encode,key2_Encode
0,109,295,4906,5854,18
1,114,1201,3733,5854,18
2,114,220,5995,5854,18
3,109,83,700,5854,18
4,109,251,4736,5854,18
...,...,...,...,...,...
21079,64,488,2839,8274,20
21080,64,488,2838,8275,20
21081,0,919,4200,8270,5
21082,5,1231,1601,8271,20


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# y에 key2가 들어있는 거임.
X_tr, X_val, y_tr, y_val = train_test_split(data_Encode[features], data_Encode[target], stratify = data_Encode[target], test_size= 0.3)

In [13]:
from pycaret.classification import *

In [14]:
# train, validation
TR = pd.concat([X_tr, y_tr], axis=1)
VAL = pd.concat([X_val, y_val], axis=1)

환경설정

In [15]:
clf= setup(data=TR, target='key2_Encode', use_gpu=True, categorical_features=['Machinery_Encode', 'Assembly_Encode', '청구품목_Encode', 'part1_Encode'])
# categorical을 안 주면 label을 숫자로 주니까 쓸 수가 없음
# clf= setup(data=TR, target='key2_Encode', use_gpu=True)

,Description,Value
0,session_id,1133
1,Target,key2_Encode
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(14758, 5)"
5,Missing Values,False
6,Numeric Features,0
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


BEST 5 출력

In [ ]:
# top5 = compare_models(sort='Accuracy', n_select=10)
# 점심시간 전에 model blending
top5 = compare_models(sort='Accuracy', n_select=5, fold=5)
blended_models = blend_models(top5, fold=5)
finalized_models = finalize_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7872,0.0000,0.6937,0.8000,0.7845,0.7695,0.7711,4.5280
svm,SVM - Linear Kernel,0.7805,0.0000,0.6801,0.7942,0.7776,0.7629,0.7639,13.4200
gbc,Gradient Boosting Classifier,0.6787,0.5494,0.6068,0.8019,0.6934,0.6411,0.6666,4154.3200
lr,Logistic Regression,0.6624,0.5678,0.4697,0.7283,0.6536,0.6259,0.6374,34.8140
et,Extra Trees Classifier,0.6543,0.5396,0.5738,0.6672,0.6524,0.6268,0.6276,18.5800
rf,Random Forest Classifier,0.6507,0.5437,0.5648,0.6695,0.6483,0.6219,0.6231,13.3940
nb,Naive Bayes,0.6455,0.4968,0.6426,0.7285,0.6697,0.6260,0.6277,17.4220
dt,Decision Tree Classifier,0.6311,0.4807,0.5465,0.6571,0.6317,0.6005,0.6020,22.8060
lda,Linear Discriminant Analysis,0.4780,0.4920,0.4438,0.6209,0.5219,0.4614,0.4647,181.2280
knn,K Neighbors Classifier,0.4227,0.4802,0.3720,0.5319,0.4438,0.3834,0.3857,94.4900


IntProgress(value=0, description='Processing: ', max=6)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:12:45
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Fold,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


In [ ]:
predictions_blend = predict_model(data=VAL, estimator=finalized_models)
predictions_blend
save_model(predictions_blend, './original_model3/blend')

In [ ]:
ridge1 = create_model('ridge')
finalize_model(ridge1)
save_model(ridge1, './original_model3/ridge1')
prediction_ridge = predict_model(ridge1, VAL)
prediction_ridge

In [ ]:
rf1 = create_model('rf')
tuned_rf1 = tune_model(rf1)
final_rf1 = finalize_model(tuned_rf1)
save_model(rf1, './original_model3/rf1')
prediction_rf = predict_model(rf1, VAL)
prediction_rf

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6360,0.0000,0.5574,0.6475,0.6293,0.6056,0.6070
1,0.6602,0.0000,0.6245,0.6779,0.6616,0.6345,0.6351
2,0.6864,0.0000,0.6119,0.7101,0.6862,0.6615,0.6625
3,0.6776,0.0000,0.6216,0.7007,0.6743,0.6511,0.6528
4,0.6525,0.0000,0.5595,0.6760,0.6511,0.6249,0.6259
5,0.6496,0.9036,0.5744,0.6671,0.6486,0.6213,0.6225
6,0.6496,0.9214,0.5856,0.6790,0.6470,0.6200,0.6219
7,0.6747,0.9220,0.6042,0.6816,0.6672,0.6476,0.6493
8,0.6631,0.9155,0.5885,0.6857,0.6559,0.6345,0.6367


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:58:01
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Fold,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.1min


In [ ]:
# 21%
ada1 = create_model('ada')
finalize_model(ada1)
save_model(ada1, './original_model3/ada1')
prediction_ada = predict_model(ada1, VAL)
prediction_ada